In [1]:
%matplotlib inline
import cv2
import matplotlib
import numpy as np
from matplotlib import pyplot as plt

plt.style.use("ggplot")

In [2]:
import FileIO as fio
import Camera as camsolve

In [3]:
root_dir = "/home/liyanc/Desktop/raid/scratch2/pose/processing/staging"
ref_intrinsic = "meta_mid/cameras/subj_take/haoyu_motion_01_camparams.pkl"
ref_dlsr = fio.load_pkl("{:}/{:}".format(root_dir,ref_intrinsic))

In [4]:
ref_e, ref_dist = ref_dlsr["03"][:2]

In [5]:
candidates = [
    ("zhe", "action_02"), ("zhe", "motion_00"), ("zhe", "motion_01"), ("lin", "zw_static_01")]

In [6]:
for sub, takename in candidates:
    corr_f = "{:}/meta_mid/cameras/subj_take/{:}_{:}_camproj.pkl".format(root_dir, sub, takename)
    save_f = "{:}/meta_mid/cameras/solvepnp_params/{:}_{:}_camparams.pkl".format(root_dir, sub, takename)
    try:
        corr = fio.load_pkl(corr_f)
    except:
        print("Failed to read file: ", sub, takename)
    else:
        new_param = {}

        for cam, corr_pairs in corr.items():
            pts, imgpts = corr_pairs
            _, rot, t, inliers = cv2.solvePnPRansac(pts, imgpts, ref_e, ref_dist, reprojectionError=0.8)
            new_param[cam] = ref_dlsr["03"][:2] + (rot, t, ref_dlsr["03"][4])
            print(sub, takename, cam, "inliers: ", len(inliers) / pts.shape[0])
            fio.dump_pkl(new_param, save_f)

zhe action_02 04 inliers:  0.4897959183673469
zhe motion_00 03 inliers:  0.25
zhe motion_01 03 inliers:  0.2857142857142857
lin zw_static_01 04 inliers:  0.4583333333333333
